<a href="https://colab.research.google.com/github/guy998877/projects/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Creating Your First QA Pipeline with Retrieval-Augmentation

- **Level**: Beginner
- **Time to complete**: 10 minutes
- **Components Used**: [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`SentenceTransformersTextEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), [`PromptBuilder`](https://docs.haystack.deepset.ai/docs/promptbuilder), [`OpenAIGenerator`](https://docs.haystack.deepset.ai/docs/openaigenerator)
- **Prerequisites**: You must have an [OpenAI API Key](https://platform.openai.com/api-keys).
- **Goal**: After completing this tutorial, you'll have learned the new prompt syntax and how to use PromptBuilder and OpenAIGenerator to build a generative question-answering pipeline with retrieval-augmentation.

> This tutorial uses Haystack 2.0. To learn more, read the [Haystack 2.0 announcement](https://haystack.deepset.ai/blog/haystack-2-release) or visit the [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/docs/intro).

## Overview

This tutorial shows you how to create a generative question-answering pipeline using the retrieval-augmentation ([RAG](https://www.deepset.ai/blog/llms-retrieval-augmentation)) approach with Haystack 2.0. The process involves four main components: [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder) for creating an embedding for the user query, [InMemoryBM25Retriever](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever) for fetching relevant documents, [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder) for creating a template prompt, and [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/openaigenerator) for generating responses.

For this tutorial, you'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents, but you can replace them with any text you want.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/logging)

## Installing Haystack

Install Haystack 2.0 and other required packages with `pip`:

In [ ]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"

### Enabling Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/enabling-telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(27)

## Fetching and Indexing Documents

You'll start creating your question answering system by downloading the data and indexing the data with its embeddings to a DocumentStore.

In this tutorial, you will take a simple approach to writing documents and their embeddings into the DocumentStore. For a full indexing pipeline with preprocessing, cleaning and splitting, check out our tutorial on [Preprocessing Different File Types](https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline).


### Initializing the DocumentStore

Initialize a DocumentStore to index your documents. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, you'll be using the `InMemoryDocumentStore`.

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store).

The DocumentStore is now ready. Now it's time to fill it with some Documents.

### Fetch the Data

You'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents. We preprocessed the data and uploaded to a Hugging Face Space: [Seven Wonders](https://huggingface.co/datasets/bilgeyucel/seven-wonders). Thus, you don't need to perform any additional cleaning or splitting.

Fetch the data and convert it into Haystack Documents:

In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset
from haystack import Document

# Load the dataset from Hugging Face
dataset = load_dataset("mvbhat/verdicts", split="train")

# Convert the dataset to a Pandas DataFrame
df = dataset.to_pandas()

# Combine the 'facts' and 'verdict' columns into a new 'content' column
df['content'] = df['facts'] + " " + df['verdict']

# Assuming df is your modified DataFrame with the 'content' column
new_dataset = dataset.from_pandas(df[['content']])

# Converting to Haystack Document objects if needed
docs = [Document(content=doc["content"]) for doc in new_dataset]


In [ ]:
df

In [ ]:
docs[0].content

IMPORT macadeliccc/US-SupremeCourtVerdicts FROM hugging face


In [ ]:
from datasets import load_dataset
from haystack import Document

# Load the dataset from Hugging Face
df_SupremeCourt = load_dataset("macadeliccc/US-SupremeCourtVerdicts", split="train")

# Convert the dataset to a Pandas DataFrame
df_SupremeCourt = df_SupremeCourt.to_pandas()


In [ ]:
df_SupremeCourt

In [ ]:
# Load the dataset from Hugging Face
dataset_supreme_court = load_dataset("macadeliccc/US-SupremeCourtVerdicts", split="train")

# Convert the dataset to a Pandas DataFrame
df_supreme_court = dataset_supreme_court.to_pandas()

# Combine the 'facts' and 'verdict' columns into a new 'content' column
df_supreme_court['content'] = df_supreme_court['category'] + " " + df_supreme_court['summary']

# Assuming df is your modified DataFrame with the 'content' column
new_dataset2 = dataset_supreme_court.from_pandas(df_supreme_court[['content']])

# Converting to Haystack Document objects if needed
docs2 = [Document(content=doc["content"]) for doc in new_dataset2]

In [ ]:
docs2[0].content

In [ ]:
docs = docs + docs2

In [ ]:
# Retrieve top documents


In [ ]:
# import math
# from collections import Counter, defaultdict

# class BM25:
#     def __init__(self, documents, k1=1.5, b=0.75):
#         self.documents = documents
#         self.k1 = k1
#         self.b = b
#         self.doc_freqs = []
#         self.doc_len = []
#         self.avgdl = 0
#         self.corpus_size = 0
#         self.idf = {}

#         self._initialize()

#     def _initialize(self):
#         # Calculate document frequencies for terms
#         df = defaultdict(int)
#         total_length = 0

#         for doc in self.documents:
#             self.corpus_size += 1
#             total_length += len(doc)
#             frequencies = Counter(doc)
#             self.doc_freqs.append(frequencies)
#             self.doc_len.append(len(doc))
#             for word in frequencies:
#                 df[word] += 1

#         self.avgdl = total_length / self.corpus_size

#         # Calculate IDF for each term
#         for word, freq in df.items():
#             self.idf[word] = math.log(1 + (self.corpus_size - freq + 0.5) / (freq + 0.5))

#     def score(self, query, index):
#         score = 0.0
#         frequencies = self.doc_freqs[index]
#         doc_length = self.doc_len[index]

#         for term in query:
#             if term in frequencies:
#                 freq = frequencies[term]
#                 idf = self.idf.get(term, 0)
#                 term_score = idf * ((freq * (self.k1 + 1)) / (freq + self.k1 * (1 - self.b + self.b * (doc_length / self.avgdl))))
#                 score += term_score

#         return score

#     def get_scores(self, query):
#         scores = []
#         for index in range(self.corpus_size):
#             scores.append(self.score(query, index))
#         return scores

#     def retrieve(self, query, top_n=5):
#         query = query.split()
#         scores = self.get_scores(query)
#         ranked_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
#         return [(self.documents[i], score) for i, score in ranked_scores[:top_n]]

# # # Example usage:
# # documents = [
# #     "The Statue of Liberty is located in New York City.",
# #     "The Colossus of Rhodes was one of the Seven Wonders of the Ancient World.",
# #     "The Great Wall of China is a historic wall in China.",
# #     "Machu Picchu is a 15th-century Inca citadel located in Peru.",
# #     "The Statue of Liberty was a gift from France to the United States."
# # ]

# # bm25 = BM25([doc.split() for doc in documents])

# # query = "Statue of Liberty gift"
# # results = bm25.retrieve(query)

# # for doc, score in results:
# #     print(f"Score: {score:.4f}, Document: {' '.join(doc)}")


In [ ]:
# Retrieve top documents
#results = bm25.retrieve(query)

In [ ]:
len(docs)

In [ ]:
from datasets import load_dataset
from haystack import Document

# dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
# docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

### Initalize a Document Embedder

To store your data in the DocumentStore with embeddings, initialize a [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) with the model name and call `warm_up()` to download the embedding model.

> If you'd like, you can use a different [Embedder](https://docs.haystack.deepset.ai/docs/embedders) for your documents.

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

### Write Documents to the DocumentStore

Run the `doc_embedder` with the Documents. The embedder will create embeddings for each document and save these embeddings in Document object's `embedding` field. Then, you can write the Documents to the DocumentStore with `write_documents()` method.

In [ ]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/66 [00:00<?, ?it/s]

## Building the RAG Pipeline

The next step is to build a [Pipeline](https://docs.haystack.deepset.ai/docs/pipelines) to generate answers for the user query following the RAG approach. To create the pipeline, you first need to initialize each component, add them to your pipeline, and connect them.

### Initialize a Text Embedder

Initialize a text embedder to create an embedding for the user query. The created embedding will later be used by the Retriever to retrieve relevant documents from the DocumentStore.

> ⚠️ Notice that you used `sentence-transformers/all-MiniLM-L6-v2` model to create embeddings for your documents before. This is why you need to use the same model to embed the user queries.

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

# english embedding
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


# Switch to DictaLM-2.0 embedding model from Hugging Face
#text_embedder = SentenceTransformersTextEmbedder(model="dicta-il/dictalm2.0")


In [ ]:

# try to dowland hebrew embedding, session crash using all availible ram

# from sentence_transformers import SentenceTransformer

# # Example text to embed
# sample_text = "This is a test sentence."

# # Using the first embedder (all-MiniLM-L6-v2)
# model1 = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# embedding1 = model1.encode(sample_text)
# print("Embedding from all-MiniLM-L6-v2:", embedding1)

# # Using the second embedder (DictaLM-2.0)
# model2 = SentenceTransformer("dicta-il/dictalm2.0")
# embedding2 = model2.encode(sample_text)
# print("Embedding from DictaLM-2.0:", embedding2)

# # Optionally, compare the embeddings using cosine similarity
# from sklearn.metrics.pairwise import cosine_similarity

# similarity = cosine_similarity([embedding1], [embedding2])
# print("Cosine Similarity between the two embeddings:", similarity[0][0])



### Initialize the Retriever

Initialize a [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever) and make it use the InMemoryDocumentStore you initialized earlier in this tutorial. This Retriever will get the relevant documents to the query.

In [ ]:
# # prompt: ModuleNotFoundError: No module named 'haystack.components.retrievers.sparse'

# !pip install farm-haystack[sparse]


In [ ]:
# ! pip uninstall pydantic
# ! pip install pydantic==1.10.9
# ! pip install --upgrade --force-reinstall haystack-ai




In [ ]:
# ! pip show haystack-ai

In [ ]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

# from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

# from haystack.nodes import ElasticsearchRetriever

# document_store = ElasticsearchDocumentStore()
# es_retriever = ElasticsearchRetriever(document_store=document_store)


#bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize the BM25 retriever
#retriever = BM25Retriever(document_store=document_store)

# from haystack.nodes import TfidfRetriever
# tfidf_retriever = TfidfRetriever(document_store=document_store)


retriever = InMemoryEmbeddingRetriever(document_store)

### Define a Template Prompt

Create a custom prompt for a generative question answering task using the RAG approach. The prompt should take in two parameters: `documents`, which are retrieved from a document store, and a `question` from the user. Use the Jinja2 looping syntax to combine the content of the retrieved documents in the prompt.

Next, initialize a [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder) instance with your prompt template. The PromptBuilder, when given the necessary values, will automatically fill in the variable values and generate a complete prompt. This approach allows for a more tailored and effective question-answering experience.

In [ ]:
# from haystack.components.builders import PromptBuilder

# template = """
# Given the following information, answer the question.

# Context:
# {% for document in documents %}
#     {{ document.content }}
# {% endfor %}

# Question: {{question}}
# Answer:
# """

# prompt_builder = PromptBuilder(template=template)

In [ ]:
from haystack.components.builders import PromptBuilder

template = """
You are a legal expert. Given the following legal context, provide a detailed and accurate response to the question.

Legal Context:
{% for document in documents %}
    - Case Summary: {{ document.content }}
{% endfor %}

Question: {{question}}

As an expert in law, consider the following:
- The minimal and maximal range of punishment imposed for the described offenses.
- Any differences in punishment based on the presence or absence of the defendant's remorse.
- The frequency with which certain circumstances occur together in similar cases.

Provide a well-reasoned answer based on the context provided:
Answer:
"""

prompt_builder = PromptBuilder(template=template)

### Initialize a Generator


Generators are the components that interact with large language models (LLMs). Now, set `OPENAI_API_KEY` environment variable and initialize a [OpenAIGenerator](https://docs.haystack.deepset.ai/docs/OpenAIGenerator) that can communicate with OpenAI GPT models. As you initialize, provide a model name:

In [ ]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-3.5-turbo")

> You can replace `OpenAIGenerator` in your pipeline with another `Generator`. Check out the full list of generators [here](https://docs.haystack.deepset.ai/docs/generators).

### Build the Pipeline

To build a pipeline, add all components to your pipeline and connect them. Create connections from `text_embedder`'s "embedding" output to "query_embedding" input of `retriever`, from `retriever` to `prompt_builder` and from `prompt_builder` to `llm`. Explicitly connect the output of `retriever` with "documents" input of the `prompt_builder` to make the connection obvious as `prompt_builder` has two inputs ("documents" and "question").

For more information on pipelines and creating connections, refer to [Creating Pipelines](https://docs.haystack.deepset.ai/docs/creating-pipelines) documentation.

In [ ]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

That's it! Your RAG pipeline is ready to generate answers to questions!

## Asking a Question

When asking a question, use the `run()` method of the pipeline. Make sure to provide the question to both the `text_embedder` and the `prompt_builder`. This ensures that the `{{question}}` variable in the template prompt gets replaced with your specific question.

In [ ]:
question = "What was the court's decision in the case involving the Idaho Probate Code and its preference for men over women in estate administration?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

In [ ]:
question = ": What was the Court's decision regarding the protection of obscene materials under the First Amendment, and how did it modify the test for obscenity?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

In [ ]:
# prompt:  No module named 'deepeval'

!pip install deepeval


In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

contextual_precision = ContextualPrecisionMetric()
contextual_recall = ContextualRecallMetric()
contextual_relevancy = ContextualRelevancyMetric()

# Load your CSV file
# Load the dataset from Hugging Face
dataset = load_dataset("mvbhat/verdicts", split="train")

# Convert the dataset to a Pandas DataFrame
df = dataset.to_pandas()

# Example questions with expected outputs and relevant contexts from the CSV
test_cases_data = [
    {
        "question": "What is the maximum punishment given in cases involving mass mailing campaigns?",
        "expected_output": "The punishment for cases involving mass mailing campaigns typically involves restrictions or penalties related to obscenity laws, with potential fines or imprisonment depending on the severity of the offense.",
        "relevant_context": df[df['facts'].str.contains("mass mailing campaign")]['facts'].iloc[0]
    },
    {
        "question": "What is the range of punishment in cases involving forgery?",
        "expected_output": "The punishment for forgery typically includes fines and imprisonment, depending on the severity and circumstances of the offense.",
        "relevant_context": df[df['facts'].str.contains("forged money orders")]['facts'].iloc[0]
    },
    {
        "question": "How often are gender-based inheritance rules upheld in court?",
        "expected_output": "Gender-based inheritance rules are rarely upheld in modern courts, particularly when they conflict with equal protection under the law.",
        "relevant_context": df[df['facts'].str.contains("Idaho Probate Code")]['facts'].iloc[0]
    },
    {
        "question": "What was the outcome of cases involving obscenity laws?",
        "expected_output": "Cases involving obscenity laws often result in convictions or penalties, with the Supreme Court affirming that obscene materials are not protected under the First Amendment.",
        "relevant_context": df[df['facts'].str.contains("obscene material")]['facts'].iloc[0]
    },
    {
        "question": "How are cases involving reproductive rights typically resolved?",
        "expected_output": "Cases involving reproductive rights often focus on privacy and personal liberty, with the courts generally protecting these rights.",
        "relevant_context": df[df['facts'].str.contains("Jane Roe")]['facts'].iloc[0]
    },
    {
        "question": "What is the range of punishments for cases involving free speech?",
        "expected_output": "Punishments in cases involving free speech can vary widely but often involve fines or imprisonment when speech is determined to incite violence or harm.",
        "relevant_context": df[df['facts'].str.contains("obscene material")]['facts'].iloc[0]
    },

    {
        "question": "How do courts rule in cases involving privacy rights?",
        "expected_output": "Courts often uphold privacy rights, particularly in matters involving personal autonomy and medical decisions.",
        "relevant_context": df[df['facts'].str.contains("Jane Roe")]['facts'].iloc[0]
    }
]

# Initialize a dictionary to store the results
results = {}



# Loop through each test case
for i, case in enumerate(test_cases_data):
    response = basic_rag_pipeline.run({"text_embedder": {"text": case["question"]}, "prompt_builder": {"question": case["question"]}})
    test_case = LLMTestCase(
        input=case["question"],
        actual_output=response["llm"]["replies"][0],
        expected_output=case["expected_output"],
        retrieval_context=[case["relevant_context"]] if case["relevant_context"] else []
    )

    # Evaluate the test case
    contextual_precision.measure(test_case)
    contextual_recall.measure(test_case)
    contextual_relevancy.measure(test_case)

    # Store the results in the dictionary
    results[f"Test_{i+1}"] = {
        "precision_score": contextual_precision.score,
        "precision_reason": contextual_precision.reason,
        "recall_score": contextual_recall.score,
        "recall_reason": contextual_recall.reason,
        "relevancy_score": contextual_relevancy.score,
        "relevancy_reason": contextual_relevancy.reason
    }

# The 'results' dictionary now contains the evaluation results for all test cases
print(results)


In [ ]:
results

In [ ]:
# Result data from the test cases
result = {
    'Test_1': {'precision_score': 1.0, 'recall_score': 0.5, 'relevancy_score': 0.0},
    'Test_2': {'precision_score': 0, 'recall_score': 0.0, 'relevancy_score': 0.0},
    'Test_3': {'precision_score': 1.0, 'recall_score': 0.0, 'relevancy_score': 0.0},
    'Test_4': {'precision_score': 1.0, 'recall_score': 0.5, 'relevancy_score': 0.0},
    'Test_5': {'precision_score': 0, 'recall_score': 0.0, 'relevancy_score': 0},
    'Test_6': {'precision_score': 1.0, 'recall_score': 0.0, 'relevancy_score': 0.0},
    'Test_7': {'precision_score': 0, 'recall_score': 0.0, 'relevancy_score': 0.0},
    'Test_8': {'precision_score': 0, 'recall_score': 0.0, 'relevancy_score': 0},
    'Test_9': {'precision_score': 1.0, 'recall_score': 0.0, 'relevancy_score': 0.0},
    'Test_10': {'precision_score': 0, 'recall_score': 0.0, 'relevancy_score': 0.0}
}

# Calculate the average scores
total_precision = sum(test['precision_score'] for test in result.values())
total_recall = sum(test['recall_score'] for test in result.values())
total_relevancy = sum(test['relevancy_score'] for test in result.values())

num_tests = len(result)

avg_precision = total_precision / num_tests
avg_recall = total_recall / num_tests
avg_relevancy = total_relevancy / num_tests

avg_precision, avg_recall, avg_relevancy

**basic prompt (0.5, 0.1, 0.0)**

Precision (0.5):  RAG pipeline is retrieving some relevant information, but not consistently across all cases.

Recall (0.1):  RAG pipeline is missing a significant amount of relevant information that should be retrieved to answer the queries comprehensively.

Relevancy (0.0): The information being retrieved is not contextually appropriate, meaning it does not directly answer or relate well to the queries.

after quesry modification (0.5, 0.1, 0.0)

**after query modification (0.5, 0.1, 0.0)**

Here are some other example questions to test:

In [ ]:
 df[df['facts'].str.contains("mass mailing campaign")]['facts'].iloc[0]

## What's next

🎉 Congratulations! You've learned how to create a generative QA system for your documents with the RAG approach.

If you liked this tutorial, you may also enjoy:
- [Filtering Documents with Metadata](https://haystack.deepset.ai/tutorials/31_metadata_filtering)
- [Preprocessing Different File Types](https://haystack.deepset.ai/tutorials/30_file_type_preprocessing_index_pipeline)
- [Creating a Hybrid Retrieval Pipeline](https://haystack.deepset.ai/tutorials/33_hybrid_retrieval)

To stay up to date on the latest Haystack developments, you can [subscribe to our newsletter](https://landing.deepset.ai/haystack-community-updates) and [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!